# Importing neccesary libraries


In [1]:
import matplotlib 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
    
    
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as data

In [2]:
data = pd.read_csv('train.csv')
data

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,...,0,0,0,0,0,0,2,0,0,Class_6
1,1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,2,0,0,0,0,0,1,0,3,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,3,0,0,7,0,1,5,2,2,0,...,0,4,0,2,2,0,4,3,0,Class_8
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199995,0,1,6,0,1,32,0,6,0,...,0,1,1,0,0,0,4,1,0,Class_6
199996,199996,0,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_6
199997,199997,1,2,0,0,0,2,0,1,8,...,1,0,1,1,1,0,1,0,0,Class_8
199998,199998,0,0,2,0,2,1,0,0,3,...,0,0,3,2,1,0,0,1,0,Class_7


In [3]:
data = data.drop(columns = "id")
data

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,6,1,0,0,0,0,7,0,...,0,0,0,0,0,0,2,0,0,Class_6
1,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,0,0,0,0,0,1,0,3,0,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,0,0,7,0,1,5,2,2,0,1,...,0,4,0,2,2,0,4,3,0,Class_8
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0,1,6,0,1,32,0,6,0,0,...,0,1,1,0,0,0,4,1,0,Class_6
199996,0,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_6
199997,1,2,0,0,0,2,0,1,8,4,...,1,0,1,1,1,0,1,0,0,Class_8
199998,0,0,2,0,2,1,0,0,3,1,...,0,0,3,2,1,0,0,1,0,Class_7


In [4]:
train_set = data.drop('target',axis=1)
train_target = data['target']
train_target

0         Class_6
1         Class_6
2         Class_2
3         Class_8
4         Class_2
           ...   
199995    Class_6
199996    Class_6
199997    Class_8
199998    Class_7
199999    Class_8
Name: target, Length: 200000, dtype: object

In [5]:
mean = train_set.mean(axis=0)
std = train_set.std(axis=0)
train_set = (train_set - mean)/std
train_set


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,-0.246766,-0.292573,0.583746,-0.171711,-0.270296,-0.277198,-0.255922,-0.301166,1.286084,-0.290630,...,0.237870,-0.272418,-0.254183,-0.237373,-0.328760,-0.252634,-0.328174,0.168271,-0.272629,-0.161008
1,-0.246766,-0.292573,-0.342670,-0.304128,-0.270296,-0.277198,-0.255922,-0.301166,-0.494484,-0.290630,...,-0.355835,0.798630,-0.254183,-0.237373,-0.328760,-0.252634,-0.328174,-0.301056,-0.179905,-0.161008
2,-0.246766,-0.292573,-0.342670,-0.304128,-0.270296,-0.083503,-0.255922,1.041147,-0.494484,-0.290630,...,1.227378,-0.272418,-0.254183,-0.237373,-0.328760,-0.045423,-0.328174,-0.301056,-0.272629,-0.161008
3,-0.246766,-0.292573,0.738149,-0.304128,0.070329,0.691278,0.250506,0.593709,-0.494484,-0.121625,...,-0.355835,-0.272418,0.302229,-0.237373,0.081271,0.161788,-0.328174,0.637599,0.005544,-0.161008
4,0.006923,-0.292573,-0.342670,-0.304128,-0.270296,-0.277198,-0.255922,-0.301166,-0.494484,-0.290630,...,-0.355835,-0.272418,-0.254183,-0.237373,-0.328760,-0.252634,-0.328174,-0.301056,-0.272629,-0.161008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,-0.246766,-0.042161,0.583746,-0.304128,0.070329,5.921053,-0.255922,2.383460,-0.494484,-0.290630,...,-0.157933,-0.272418,-0.115080,0.023370,-0.328760,-0.252634,-0.328174,0.637599,-0.179905,-0.161008
199996,-0.246766,0.208252,-0.342670,-0.304128,0.070329,-0.277198,-0.255922,-0.301166,-0.494484,-0.290630,...,-0.355835,-0.272418,-0.254183,-0.237373,-0.328760,-0.252634,-0.328174,-0.301056,-0.272629,-0.161008
199997,0.006923,0.208252,-0.342670,-0.304128,-0.270296,0.110192,-0.255922,0.146272,1.540451,0.385389,...,0.435772,0.263106,-0.254183,0.023370,-0.123744,-0.045423,-0.328174,-0.066392,-0.272629,-0.161008
199998,-0.246766,-0.292573,-0.033864,-0.304128,0.410953,-0.083503,-0.255922,-0.301166,0.268616,-0.121625,...,-0.355835,-0.272418,-0.254183,0.544858,0.081271,-0.045423,-0.328174,-0.301056,-0.179905,-0.161008


In [6]:
train_target = pd.get_dummies(train_target, columns = "target",dtype = int)
train_target

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0
2,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
199995,0,0,0,0,0,1,0,0,0
199996,0,0,0,0,0,1,0,0,0
199997,0,0,0,0,0,0,0,1,0
199998,0,0,0,0,0,0,1,0,0


In [25]:
train_set = train_set.values
train_target = train_target.values

AttributeError: 'builtin_function_or_method' object has no attribute 'values'

In [8]:
train_target.shape

(200000, 9)

In [9]:
train_target

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [10]:
train_set = torch.from_numpy(train_set)
train_target = torch.from_numpy(train_target)
train_set

tensor([[-0.2468, -0.2926,  0.5837,  ...,  0.1683, -0.2726, -0.1610],
        [-0.2468, -0.2926, -0.3427,  ..., -0.3011, -0.1799, -0.1610],
        [-0.2468, -0.2926, -0.3427,  ..., -0.3011, -0.2726, -0.1610],
        ...,
        [ 0.0069,  0.2083, -0.3427,  ..., -0.0664, -0.2726, -0.1610],
        [-0.2468, -0.2926, -0.0339,  ..., -0.3011, -0.1799, -0.1610],
        [ 1.0217,  0.7091, -0.3427,  ...,  0.1683,  0.0055,  0.0937]],
       dtype=torch.float64)

In [11]:
train_target1 = train_target.tolist()
il = [i for r in train_target1  for i in range(9) if r[i] == 1]

train_target3 = torch.tensor(il, dtype = torch.long)
il

[5,
 5,
 1,
 7,
 1,
 7,
 5,
 2,
 1,
 7,
 7,
 7,
 0,
 7,
 5,
 4,
 5,
 5,
 7,
 5,
 2,
 2,
 7,
 7,
 7,
 5,
 7,
 7,
 7,
 5,
 2,
 7,
 7,
 5,
 1,
 1,
 6,
 2,
 5,
 7,
 5,
 6,
 5,
 7,
 1,
 7,
 7,
 5,
 5,
 7,
 5,
 6,
 7,
 5,
 0,
 2,
 8,
 5,
 5,
 8,
 6,
 5,
 2,
 6,
 8,
 5,
 1,
 4,
 5,
 8,
 5,
 6,
 5,
 5,
 2,
 0,
 8,
 7,
 1,
 7,
 2,
 7,
 5,
 5,
 2,
 1,
 5,
 6,
 5,
 0,
 7,
 5,
 1,
 7,
 5,
 1,
 8,
 8,
 5,
 1,
 7,
 7,
 1,
 6,
 2,
 1,
 5,
 0,
 7,
 6,
 1,
 7,
 5,
 5,
 3,
 5,
 7,
 8,
 1,
 5,
 1,
 8,
 5,
 5,
 2,
 6,
 5,
 5,
 5,
 7,
 8,
 7,
 5,
 8,
 1,
 2,
 3,
 7,
 5,
 6,
 7,
 7,
 8,
 7,
 5,
 8,
 5,
 0,
 5,
 7,
 8,
 8,
 8,
 5,
 6,
 7,
 7,
 7,
 5,
 5,
 6,
 5,
 8,
 7,
 1,
 1,
 7,
 5,
 1,
 5,
 7,
 5,
 8,
 5,
 6,
 7,
 8,
 5,
 8,
 7,
 5,
 5,
 6,
 6,
 1,
 6,
 1,
 6,
 1,
 5,
 1,
 2,
 6,
 5,
 1,
 6,
 3,
 8,
 5,
 5,
 7,
 7,
 1,
 7,
 5,
 5,
 7,
 7,
 1,
 8,
 5,
 7,
 5,
 8,
 7,
 6,
 0,
 8,
 7,
 7,
 5,
 8,
 7,
 7,
 7,
 7,
 7,
 4,
 5,
 2,
 2,
 7,
 6,
 1,
 5,
 5,
 5,
 6,
 8,
 8,
 2,
 3,
 2,
 7,
 2,
 2,
 7,
 1,
 8,
 5,


In [12]:
train_target3

tensor([5, 5, 1,  ..., 7, 6, 7])

In [13]:
train_target.shape


torch.Size([200000, 9])

In [14]:

from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(75, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 9)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        
        return x

In [20]:
model = Classifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)
losses = []
for i in range(1000):
    
    optimizer.zero_grad()
    output = model(train_set.float())

    loss = criterion(output,train_target3)
    loss.backward()
    optimizer.step()
    losses.append(loss)
    if(i%100 == 0):
        print(f'Loss at {i}th epoch:')
        print(loss)


Loss at 0th epoch:
tensor(2.1801, grad_fn=<NllLossBackward>)
Loss at 100th epoch:
tensor(1.7779, grad_fn=<NllLossBackward>)
Loss at 200th epoch:
tensor(1.7546, grad_fn=<NllLossBackward>)
Loss at 300th epoch:
tensor(1.7453, grad_fn=<NllLossBackward>)
Loss at 400th epoch:
tensor(1.7396, grad_fn=<NllLossBackward>)
Loss at 500th epoch:
tensor(1.7357, grad_fn=<NllLossBackward>)
Loss at 600th epoch:
tensor(1.7323, grad_fn=<NllLossBackward>)
Loss at 700th epoch:
tensor(1.7296, grad_fn=<NllLossBackward>)
Loss at 800th epoch:
tensor(1.7274, grad_fn=<NllLossBackward>)
Loss at 900th epoch:
tensor(1.7256, grad_fn=<NllLossBackward>)


In [79]:
test_data = pd.read_csv("test.csv")
test_data

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,200000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,200001,1,2,0,0,0,0,0,0,0,...,3,1,3,0,0,0,0,3,0,0
2,200002,0,1,7,1,0,0,0,0,6,...,3,0,0,0,0,3,0,2,0,0
3,200003,0,0,0,4,3,1,0,0,0,...,0,0,0,1,0,0,0,4,0,0
4,200004,0,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,299995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99996,299996,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,299997,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
99998,299998,0,0,0,0,2,2,0,0,1,...,1,0,0,0,0,1,0,1,0,0


In [80]:
td_id = test_data['id']
td = test_data.drop('id',axis = 1)
td

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,0,0,0,0,0,0,0,...,3,1,3,0,0,0,0,3,0,0
2,0,1,7,1,0,0,0,0,6,0,...,3,0,0,0,0,3,0,2,0,0
3,0,0,0,4,3,1,0,0,0,0,...,0,0,0,1,0,0,0,4,0,0
4,0,0,5,0,0,0,0,0,0,8,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99996,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
99998,0,0,0,0,2,2,0,0,1,0,...,1,0,0,0,0,1,0,1,0,0


In [81]:
mean = td.mean(axis=0)
std = td.std(axis=0)
td = (td - mean)/std
td

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,-0.246461,-0.289939,-0.340901,-0.305382,-0.269553,-0.277977,-0.257023,-0.300435,-0.493949,-0.292018,...,-0.354183,-0.271905,-0.253651,-0.239527,-0.326833,-0.251128,-0.328936,-0.295545,-0.272067,-0.161275
1,0.002361,0.206311,-0.340901,-0.305382,-0.269553,-0.277977,-0.257023,-0.300435,-0.493949,-0.292018,...,0.236913,0.261975,0.170595,-0.239527,-0.326833,-0.251128,-0.328936,0.379544,-0.272067,-0.161275
2,-0.246461,-0.041814,0.752367,-0.173538,-0.269553,-0.277977,-0.257023,-0.300435,1.029510,-0.292018,...,0.236913,-0.271905,-0.253651,-0.239527,-0.326833,0.375913,-0.328936,0.154514,-0.272067,-0.161275
3,-0.246461,-0.289939,-0.340901,0.221993,0.751416,-0.084335,-0.257023,-0.300435,-0.493949,-0.292018,...,-0.354183,-0.271905,-0.253651,0.023788,-0.326833,-0.251128,-0.328936,0.604574,-0.272067,-0.161275
4,-0.246461,-0.289939,0.440005,-0.305382,-0.269553,-0.277977,-0.257023,-0.300435,-0.493949,1.069293,...,-0.354183,-0.271905,-0.253651,-0.239527,-0.326833,-0.251128,-0.328936,-0.295545,-0.179374,-0.161275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-0.246461,-0.289939,-0.340901,-0.305382,-0.269553,-0.277977,-0.257023,-0.300435,-0.493949,-0.292018,...,-0.354183,-0.271905,-0.253651,-0.239527,-0.326833,-0.251128,-0.328936,-0.295545,-0.272067,-0.161275
99996,-0.246461,-0.289939,-0.340901,-0.305382,-0.269553,-0.084335,-0.257023,-0.300435,-0.493949,-0.292018,...,-0.354183,-0.271905,-0.253651,-0.239527,-0.326833,-0.251128,-0.328936,-0.295545,-0.272067,-0.161275
99997,-0.246461,-0.289939,-0.340901,-0.305382,-0.269553,-0.277977,-0.257023,-0.300435,-0.493949,-0.292018,...,-0.354183,-0.271905,-0.112236,-0.239527,-0.326833,-0.251128,-0.328936,-0.295545,-0.272067,0.341069
99998,-0.246461,-0.289939,-0.340901,-0.305382,0.411093,0.109307,-0.257023,-0.300435,-0.240039,-0.292018,...,-0.157151,-0.271905,-0.253651,-0.239527,-0.326833,-0.042114,-0.328936,-0.070515,-0.272067,-0.161275


In [82]:
td = td.values
td1 = torch.from_numpy(td)
td1

tensor([[-0.2465, -0.2899, -0.3409,  ..., -0.2955, -0.2721, -0.1613],
        [ 0.0024,  0.2063, -0.3409,  ...,  0.3795, -0.2721, -0.1613],
        [-0.2465, -0.0418,  0.7524,  ...,  0.1545, -0.2721, -0.1613],
        ...,
        [-0.2465, -0.2899, -0.3409,  ..., -0.2955, -0.2721,  0.3411],
        [-0.2465, -0.2899, -0.3409,  ..., -0.0705, -0.2721, -0.1613],
        [-0.2465, -0.2899, -0.3409,  ..., -0.2955, -0.2721, -0.1613]],
       dtype=torch.float64)

In [83]:
predictions = model(td1.float())
predictions


tensor([[-2.6599, -1.0105, -1.8793,  ..., -3.6791, -2.9199, -2.0754],
        [-3.4434, -3.0518, -3.2428,  ..., -2.5769, -0.9933, -2.2773],
        [-3.8914, -3.8615, -4.0223,  ..., -3.4484, -1.9241, -3.1646],
        ...,
        [-2.7120, -1.1905, -1.9658,  ..., -3.0615, -2.2346, -1.8984],
        [-3.1287, -3.8086, -3.7684,  ..., -2.4915, -1.2132, -2.5811],
        [-3.3774, -3.1401, -3.0403,  ..., -2.7108, -1.3942, -2.2154]],
       grad_fn=<LogSoftmaxBackward>)

In [84]:
pred_np = predictions.detach().numpy()
pred_np

array([[-2.6598828 , -1.0105151 , -1.879272  , ..., -3.6791382 ,
        -2.9198735 , -2.075437  ],
       [-3.4433866 , -3.0518203 , -3.2428465 , ..., -2.576895  ,
        -0.99334854, -2.2773392 ],
       [-3.8913755 , -3.861536  , -4.022332  , ..., -3.448426  ,
        -1.9241155 , -3.1645591 ],
       ...,
       [-2.7119994 , -1.1904635 , -1.9657508 , ..., -3.0615127 ,
        -2.234562  , -1.8983514 ],
       [-3.1287212 , -3.808641  , -3.768401  , ..., -2.4914966 ,
        -1.2131778 , -2.5810795 ],
       [-3.3773608 , -3.1401143 , -3.040349  , ..., -2.710798  ,
        -1.3941981 , -2.215426  ]], dtype=float32)

In [85]:
pred_df = pd.DataFrame(pred_np)
pred_df

,0,1,2,3,4,5,6,7,8
0,-2.659883,-1.010515,-1.879272,-3.616168,-4.259809,-1.786095,-3.679138,-2.919873,-2.075437
1,-3.443387,-3.051820,-3.242846,-4.613463,-5.133662,-1.148838,-2.576895,-0.993349,-2.277339
2,-3.891376,-3.861536,-4.022332,-4.755300,-5.212954,-0.347360,-3.448426,-1.924116,-3.164559
3,-3.227212,-2.526116,-2.753390,-3.501282,-4.188432,-1.100314,-2.435133,-1.492156,-2.070815
4,-3.722386,-2.746600,-3.308812,-4.264089,-4.633281,-0.792621,-3.194512,-1.442426,-2.109205
...,...,...,...,...,...,...,...,...,...
99995,-2.716533,-1.086605,-1.759535,-3.576882,-4.373095,-2.323784,-3.152593,-2.378830,-1.893564
99996,-2.938074,-1.522102,-2.060855,-3.610686,-4.134617,-1.699954,-2.917261,-1.916706,-1.745668
99997,-2.711999,-1.190464,-1.965751,-3.440207,-4.181797,-1.977139,-3.061513,-2.234562,-1.898351
99998,-3.128721,-3.808641,-3.768401,-4.238211,-4.456953,-0.845836,-2.491497,-1.213178,-2.581079


In [86]:
td_id.columns = ['id']
td_id = pd.DataFrame(td_id)
td_id = td_id.to_numpy()
td_id


array([[200000],
       [200001],
       [200002],
       ...,
       [299997],
       [299998],
       [299999]], dtype=int64)

In [87]:
from sklearn import preprocessing

x = pred_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
pred_df = pd.DataFrame(x_scaled)
pred_df

,0,1,2,3,4,5,6,7,8
0,0.756452,0.985083,0.836454,0.717349,0.738928,0.709935,0.563369,0.411144,0.943263
1,0.644628,0.921103,0.654243,0.581789,0.622846,0.814210,0.700132,0.814072,0.936510
2,0.580689,0.895724,0.550083,0.562509,0.612313,0.945357,0.591995,0.619404,0.906836
3,0.675481,0.937580,0.719648,0.732965,0.748409,0.822150,0.717722,0.709747,0.943418
4,0.604808,0.930670,0.645428,0.629278,0.689316,0.872498,0.623500,0.720148,0.942134
...,...,...,...,...,...,...,...,...,...
99995,0.748366,0.982699,0.852454,0.722689,0.723879,0.621952,0.628701,0.524302,0.949346
99996,0.716747,0.969049,0.812190,0.718094,0.755558,0.724030,0.657901,0.620954,0.954293
99997,0.749014,0.979443,0.824898,0.741267,0.749291,0.678674,0.640002,0.554475,0.949186
99998,0.689538,0.897382,0.584015,0.632796,0.712739,0.863791,0.710728,0.768095,0.926351


In [88]:
summ = pred_df.sum(axis = 1)
for i in range(100000):
    pred_df.iloc[i] = pred_df.iloc[i]/summ.iloc[i] 


In [89]:
pred_df

,0,1,2,3,4,5,6,7,8
0,0.113548,0.147867,0.125556,0.107678,0.110917,0.106565,0.084565,0.061715,0.141589
1,0.096364,0.137693,0.097801,0.086970,0.093108,0.121714,0.104661,0.121693,0.139996
2,0.092689,0.142975,0.087804,0.089787,0.097737,0.150897,0.094494,0.098869,0.144748
3,0.096399,0.133804,0.102702,0.104603,0.106807,0.117331,0.102427,0.101289,0.134637
4,0.090842,0.139787,0.096943,0.094518,0.103535,0.131049,0.093650,0.108166,0.141509
...,...,...,...,...,...,...,...,...,...
99995,0.110797,0.145490,0.126207,0.106995,0.107172,0.092081,0.093080,0.077624,0.140552
99996,0.103444,0.139858,0.117219,0.103639,0.109046,0.104495,0.094951,0.089619,0.137728
99997,0.109086,0.142646,0.120138,0.107958,0.109127,0.098842,0.093210,0.080754,0.138239
99998,0.101620,0.132251,0.086069,0.093258,0.105040,0.127301,0.104743,0.113198,0.136521


In [90]:
pred_df.insert(0,'id',td_id,True)
# df.insert(2, "Age", [21, 23, 24, 21], True)
pred_df

,id,0,1,2,3,4,5,6,7,8
0,200000,0.113548,0.147867,0.125556,0.107678,0.110917,0.106565,0.084565,0.061715,0.141589
1,200001,0.096364,0.137693,0.097801,0.086970,0.093108,0.121714,0.104661,0.121693,0.139996
2,200002,0.092689,0.142975,0.087804,0.089787,0.097737,0.150897,0.094494,0.098869,0.144748
3,200003,0.096399,0.133804,0.102702,0.104603,0.106807,0.117331,0.102427,0.101289,0.134637
4,200004,0.090842,0.139787,0.096943,0.094518,0.103535,0.131049,0.093650,0.108166,0.141509
...,...,...,...,...,...,...,...,...,...,...
99995,299995,0.110797,0.145490,0.126207,0.106995,0.107172,0.092081,0.093080,0.077624,0.140552
99996,299996,0.103444,0.139858,0.117219,0.103639,0.109046,0.104495,0.094951,0.089619,0.137728
99997,299997,0.109086,0.142646,0.120138,0.107958,0.109127,0.098842,0.093210,0.080754,0.138239
99998,299998,0.101620,0.132251,0.086069,0.093258,0.105040,0.127301,0.104743,0.113198,0.136521


In [91]:
pred_df.to_csv("KaggleSubmission.csv")